In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os


Turbofun Data:

In [3]:
train_file = 'train_FD001.txt'
index_names = ['unit_nr', 'time_cycles']
setting_names = ['setting_1', 'setting_2', 'setting_3']
sensor_names = ['s_{}'.format(i) for i in range(1,22)] 
col_names = index_names + setting_names + sensor_names

# read data
train = pd.read_csv((train_file), sep='\s+', header=None, 
                 names=col_names)


train.describe()


,unit_nr,time_cycles,setting_1,setting_2,setting_3,s_1,s_2,s_3,s_4,s_5,...,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20,s_21
count,20631.000000,20631.000000,20631.000000,20631.000000,20631.0,2.063100e+04,20631.000000,20631.000000,20631.000000,2.063100e+04,...,20631.000000,20631.000000,20631.000000,20631.000000,2.063100e+04,20631.000000,20631.0,20631.0,20631.000000,20631.000000
mean,51.506568,108.807862,-0.000009,0.000002,100.0,5.186700e+02,642.680934,1590.523119,1408.933782,1.462000e+01,...,521.413470,2388.096152,8143.752722,8.442146,3.000000e-02,393.210654,2388.0,100.0,38.816271,23.289705
std,29.227633,68.880990,0.002187,0.000293,0.0,6.537152e-11,0.500053,6.131150,9.000605,3.394700e-12,...,0.737553,0.071919,19.076176,0.037505,1.556432e-14,1.548763,0.0,0.0,0.180746,0.108251
min,1.000000,1.000000,-0.008700,-0.000600,100.0,5.186700e+02,641.210000,1571.040000,1382.250000,1.462000e+01,...,518.690000,2387.880000,8099.940000,8.324900,3.000000e-02,388.000000,2388.0,100.0,38.140000,22.894200
25%,26.000000,52.000000,-0.001500,-0.000200,100.0,5.186700e+02,642.325000,1586.260000,1402.360000,1.462000e+01,...,520.960000,2388.040000,8133.245000,8.414900,3.000000e-02,392.000000,2388.0,100.0,38.700000,23.221800
50%,52.000000,104.000000,0.000000,0.000000,100.0,5.186700e+02,642.640000,1590.100000,1408.040000,1.462000e+01,...,521.480000,2388.090000,8140.540000,8.438900,3.000000e-02,393.000000,2388.0,100.0,38.830000,23.297900
75%,77.000000,156.000000,0.001500,0.000300,100.0,5.186700e+02,643.000000,1594.380000,1414.555000,1.462000e+01,...,521.950000,2388.140000,8148.310000,8.465600,3.000000e-02,394.000000,2388.0,100.0,38.950000,23.366800
max,100.000000,362.000000,0.008700,0.000600,100.0,5.186700e+02,644.530000,1616.910000,1441.490000,1.462000e+01,...,523.380000,2388.560000,8293.720000,8.584800,3.000000e-02,400.000000,2388.0,100.0,39.430000,23.618400


Drop those with std equal to zero (constant values)

In [4]:
#for f0001
train=train.drop(['s_1', 's_5','s_10','s_16','s_18','s_19','setting_1','setting_2','setting_3'],axis=1)

Group by unit (create dataframe for each turbofun run-to-failure)

In [5]:
grouped_by_unit = train.groupby(by="unit_nr")
max_cycle = grouped_by_unit["time_cycles"].max()
max_cycle.max()

import datetime


dates=[]
for cycle,unit_nr in zip(train['time_cycles'],train['unit_nr']):
    dates.append( (datetime.datetime(2000, 1, 1) + datetime.timedelta(days=cycle)+ datetime.timedelta(seconds=unit_nr)) )

train['Artficial_timestamp']=dates
train.head()
grouped=train.groupby(by="unit_nr")

dfs=[grouped.get_group(x) for x in grouped.groups]
print(len(dfs))


100


Plot histogram with the end of life cycle

In [6]:
%matplotlib qt

# create empty list
dataframes_list = []
 
# append datasets to the list
endes=[]
endswithIndex=[]
for i in range(len(dfs)):
    temp_df = dfs[i]
    dataframes_list.append(temp_df)
    endswithIndex.append((i,len(temp_df.index)))
    endes.append(len(temp_df.index))
print(endswithIndex)
# fixed bin size
bins = np.arange(0, 400, 5) # fixed bin size

plt.xlim([min(endes)-100, max(endes)+100])

plt.hist(endes, bins=bins, alpha=0.5)
plt.show()

[(0, 192), (1, 287), (2, 179), (3, 189), (4, 269), (5, 188), (6, 259), (7, 150), (8, 201), (9, 222), (10, 240), (11, 170), (12, 163), (13, 180), (14, 207), (15, 209), (16, 276), (17, 195), (18, 158), (19, 234), (20, 195), (21, 202), (22, 168), (23, 147), (24, 230), (25, 199), (26, 156), (27, 165), (28, 163), (29, 194), (30, 234), (31, 191), (32, 200), (33, 195), (34, 181), (35, 158), (36, 170), (37, 194), (38, 128), (39, 188), (40, 216), (41, 196), (42, 207), (43, 192), (44, 158), (45, 256), (46, 214), (47, 231), (48, 215), (49, 198), (50, 213), (51, 213), (52, 195), (53, 257), (54, 193), (55, 275), (56, 137), (57, 147), (58, 231), (59, 172), (60, 185), (61, 180), (62, 174), (63, 283), (64, 153), (65, 202), (66, 313), (67, 199), (68, 362), (69, 137), (70, 208), (71, 213), (72, 213), (73, 166), (74, 229), (75, 210), (76, 154), (77, 231), (78, 199), (79, 185), (80, 240), (81, 214), (82, 293), (83, 267), (84, 188), (85, 278), (86, 178), (87, 213), (88, 217), (89, 154), (90, 135), (91, 341

Sort each run-to-failure, based on the end of life cycle.

We keep the index of each run-to-failure example in the first position of tuple


In [7]:
endswithIndex=sorted(endswithIndex, key=lambda tup: tup[1])
endes.sort()
print(endswithIndex)

[(38, 128), (90, 135), (56, 137), (69, 137), (23, 147), (57, 147), (7, 150), (64, 153), (76, 154), (89, 154), (92, 155), (26, 156), (97, 156), (18, 158), (35, 158), (44, 158), (12, 163), (28, 163), (27, 165), (73, 166), (22, 168), (11, 170), (36, 170), (59, 172), (62, 174), (86, 178), (2, 179), (13, 180), (61, 180), (34, 181), (60, 185), (79, 185), (98, 185), (5, 188), (39, 188), (84, 188), (3, 189), (31, 191), (0, 192), (43, 192), (54, 193), (29, 194), (37, 194), (17, 195), (20, 195), (33, 195), (52, 195), (41, 196), (49, 198), (25, 199), (67, 199), (78, 199), (32, 200), (99, 200), (8, 201), (21, 202), (65, 202), (96, 202), (14, 207), (42, 207), (70, 208), (15, 209), (75, 210), (50, 213), (51, 213), (71, 213), (72, 213), (87, 213), (46, 214), (81, 214), (48, 215), (40, 216), (88, 217), (9, 222), (74, 229), (24, 230), (47, 231), (58, 231), (77, 231), (19, 234), (30, 234), (10, 240), (80, 240), (45, 256), (53, 257), (93, 258), (6, 259), (83, 267), (4, 269), (55, 275), (16, 276), (85, 27

Cluster run-to-failure examples to step (5) bins. As desribe in the paper.


In [8]:
step=5
Buildtuples=[]
Buildtuplesindexes=[]
for i in range(int(len(endes)/step)):
    temptuple=[]
    temptupleindexes=[]
    for j in range(step):
        temptuple.append(endes[i+int(len(endes)/step)*j])
        temptupleindexes.append(i+int(len(endes)/step)*j)
    Buildtuples.append(temptuple)
    Buildtuplesindexes.append(temptupleindexes)
    print(temptuple)
Buildtuples2=[]
Buildtuplesindexes2=[]
for i in range(int(len(Buildtuples)/step)):
    temptuple=[]
    temptupleindexes=[]
    for j in range(step):
        temptuple.append(Buildtuples[i+int(len(Buildtuples)/step)*j])
        temptupleindexes.append(Buildtuplesindexes[i+int(len(Buildtuples)/step)*j])
    Buildtuples2.append(temptuple)
    Buildtuplesindexes2.append(temptupleindexes)
    print(temptuple)
    print(temptupleindexes)

[128, 168, 193, 208, 234]
[135, 170, 194, 209, 240]
[137, 170, 194, 210, 240]
[137, 172, 195, 213, 256]
[147, 174, 195, 213, 257]
[147, 178, 195, 213, 258]
[150, 179, 195, 213, 259]
[153, 180, 196, 213, 267]
[154, 180, 198, 214, 269]
[154, 181, 199, 214, 275]
[155, 185, 199, 215, 276]
[156, 185, 199, 216, 278]
[156, 185, 200, 217, 283]
[158, 188, 200, 222, 283]
[158, 188, 201, 229, 287]
[158, 188, 202, 230, 293]
[163, 189, 202, 231, 313]
[163, 191, 202, 231, 336]
[165, 192, 207, 231, 341]
[166, 192, 207, 234, 362]
[[128, 168, 193, 208, 234], [147, 174, 195, 213, 257], [154, 180, 198, 214, 269], [156, 185, 200, 217, 283], [163, 189, 202, 231, 313]]
[[0, 20, 40, 60, 80], [4, 24, 44, 64, 84], [8, 28, 48, 68, 88], [12, 32, 52, 72, 92], [16, 36, 56, 76, 96]]
[[135, 170, 194, 209, 240], [147, 178, 195, 213, 258], [154, 181, 199, 214, 275], [158, 188, 200, 222, 283], [163, 191, 202, 231, 336]]
[[1, 21, 41, 61, 81], [5, 25, 45, 65, 85], [9, 29, 49, 69, 89], [13, 33, 53, 73, 93], [17, 37, 57, 7

Flatten bins.

In [9]:
Buildtuples3=Buildtuples2
flat_list1 = [item for sublist in Buildtuples3[0] for item in sublist]# for item2 in item]
#print(Buildtuplesindexes2)
flat_listindexes1 = [item for sublist in Buildtuplesindexes2[0] for item in sublist]# for item2 in item]
print(flat_listindexes1)
flat_list1.sort()
#print(flat_list1)
pairlist1=[(0,i) for i in flat_list1]
#print(pairlist1)

[0, 20, 40, 60, 80, 4, 24, 44, 64, 84, 8, 28, 48, 68, 88, 12, 32, 52, 72, 92, 16, 36, 56, 76, 96]


Method to calculate the good propotion reference

In [10]:
def refporpotion(flat_list1):
    normalrefs=[]
    #print(endes)
    mindist=60
    for item in flat_list1:
        counter=0
        for end in flat_list1:
            if end[0]<=item[1] and end[1]-item[1]>mindist:
                counter+=1
        normalrefs.append(counter)

    badrefs=[]
    for item in flat_list1:
        counter=0
        for end in flat_list1:
            if end[0]<=item[1] and   abs(end[1]-item[1])<=mindist:
                counter+=1
        badrefs.append(counter)

    dif_bad_normal=[]
    for i in range(len(badrefs)):
        if normalrefs[i]==0:
            good_bad_porpotion=0
        else:
            good_bad_porpotion=normalrefs[i]/(normalrefs[i]+badrefs[i])
        dif_bad_normal.append(good_bad_porpotion)
    return dif_bad_normal

calculate the good propotion reference of first bin

In [11]:
normalrefs=[]
flat_list1=pairlist1
#print(endes)
mindist=60
for item in flat_list1:
    counter=0
    for end in flat_list1:
        if end[0]<=item[1] and end[1]-item[1]>mindist:
            counter+=1
    normalrefs.append(counter)
print(normalrefs)

badrefs=[]
for item in flat_list1:
    counter=0
    for end in flat_list1:
        if end[0]<=item[1] and abs(end[1]-item[1])<=mindist:
            counter+=1
    badrefs.append(counter)
print(badrefs)

dif_bad_normal=[]
for i in range(len(badrefs)):
    if normalrefs[i]==0:
        good_bad_porpotion=0
    else:
        good_bad_porpotion=normalrefs[i]/(normalrefs[i]+badrefs[i])
    dif_bad_normal.append(good_bad_porpotion)
print(dif_bad_normal)

[16, 10, 7, 7, 6, 6, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 2, 2, 2, 1, 1, 0, 0, 0, 0]
[9, 15, 18, 18, 19, 19, 21, 21, 21, 20, 20, 20, 21, 21, 21, 20, 21, 21, 19, 18, 18, 13, 9, 6, 4]
[0.64, 0.4, 0.28, 0.28, 0.24, 0.24, 0.16, 0.16, 0.16, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.125, 0.125, 0.125, 0.13043478260869565, 0.08695652173913043, 0.08695652173913043, 0.09523809523809523, 0.05263157894736842, 0.05263157894736842, 0, 0, 0, 0]


Build tuples with start cycle end end cycle for each run to failure example. These will be used to store the shift in time of each example.

In [12]:
Buildtuples3=Buildtuples2
reffs=[]
reffsindexes=[]

for lllist in Buildtuples3[1:]:
    reffs.extend(lllist)

for lllist in Buildtuplesindexes2[1:]:
    reffsindexes.extend(lllist)


flat_list2 = [item for sublist in reffs for item in sublist]# for item2 in item]
flat_list2.sort()
flat_listindexes2 = [item for sublist in reffsindexes for item in sublist]# for item2 in item]
print(flat_listindexes2)
#print(flat_list2)
pairlist2=[(0,i) for i in flat_list2]
print(pairlist2)


[1, 21, 41, 61, 81, 5, 25, 45, 65, 85, 9, 29, 49, 69, 89, 13, 33, 53, 73, 93, 17, 37, 57, 77, 97, 2, 22, 42, 62, 82, 6, 26, 46, 66, 86, 10, 30, 50, 70, 90, 14, 34, 54, 74, 94, 18, 38, 58, 78, 98, 3, 23, 43, 63, 83, 7, 27, 47, 67, 87, 11, 31, 51, 71, 91, 15, 35, 55, 75, 95, 19, 39, 59, 79, 99]
[(0, 135), (0, 137), (0, 137), (0, 147), (0, 150), (0, 153), (0, 154), (0, 155), (0, 156), (0, 158), (0, 158), (0, 158), (0, 163), (0, 165), (0, 166), (0, 170), (0, 170), (0, 172), (0, 178), (0, 179), (0, 180), (0, 181), (0, 185), (0, 185), (0, 188), (0, 188), (0, 188), (0, 191), (0, 192), (0, 192), (0, 194), (0, 194), (0, 195), (0, 195), (0, 195), (0, 196), (0, 199), (0, 199), (0, 199), (0, 200), (0, 201), (0, 202), (0, 202), (0, 207), (0, 207), (0, 209), (0, 210), (0, 213), (0, 213), (0, 213), (0, 213), (0, 214), (0, 215), (0, 216), (0, 222), (0, 229), (0, 230), (0, 231), (0, 231), (0, 234), (0, 240), (0, 240), (0, 256), (0, 258), (0, 259), (0, 267), (0, 275), (0, 276), (0, 278), (0, 283), (0, 2

In iterative procces, try to build a fleet dataset by shifting in time the examples bins.

In [13]:
goodlimit=0.65
while len(pairlist2)>0:
    targetpos=-1
    c=0
    porpotion=refporpotion(pairlist1)
    for item in porpotion:
        if item<goodlimit:
            targetpos=c
            break
        c+=1
    if targetpos==-1:
        break
    targettuple=pairlist1[targetpos]
    goodrefs=normalrefs[targetpos]
    badref=badrefs[targetpos]
    toextend=[]
    used=[]
    isDone=False
    i=0
    while len(pairlist2)>i and isDone==False:
        temp=pairlist2[i]
        shift=targettuple[0]+int(2*targettuple[1]/3)
        temp=(temp[0]+shift,temp[1]+shift)
        if temp[0]<targettuple[1] and temp[1]-targettuple[1]>mindist:
            goodrefs+=1
            toextend.append(temp)
            used.append(i)
        if goodrefs/(goodrefs+badref)>goodlimit:
            isDone=True
        i+=1
    newpairlist1=[]
    newpairlist1.extend(pairlist1)
    newpairlist1.extend(toextend)
    print(used)
    newpairlist2=[]
    for i in range(len(pairlist2)):
        if i not in used:
            newpairlist2.append(pairlist2[i])
    pairlist1=newpairlist1
    pairlist2=newpairlist2

[0]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]


In [14]:
dif_bad_normal=refporpotion(pairlist1)
print(dif_bad_normal)
print(pairlist1)
flat_listindexes1.extend(flat_listindexes2)
PairIndexes=flat_listindexes1
print(PairIndexes)

[0.91, 0.85, 0.82, 0.82, 0.8, 0.8, 0.78, 0.76, 0.75, 0.7474747474747475, 0.7171717171717171, 0.7070707070707071, 0.6666666666666666, 0.6666666666666666, 0.6565656565656566, 0.6224489795918368, 0.6020408163265306, 0.6020408163265306, 0.6041666666666666, 0.5212765957446809, 0.48936170212765956, 0.32954545454545453, 0.34523809523809523, 0.2625, 0.18055555555555555, 0.6041666666666666, 0.4946236559139785, 0.4946236559139785, 0.358695652173913, 0.3626373626373626, 0.3333333333333333, 0.3333333333333333, 0.32222222222222224, 0.3258426966292135, 0.32954545454545453, 0.32954545454545453, 0.32954545454545453, 0.3372093023255814, 0.3411764705882353, 0.3411764705882353, 0.3411764705882353, 0.3411764705882353, 0.34523809523809523, 0.2926829268292683, 0.2625, 0.2625, 0.2625, 0.25, 0.25, 0.25, 0.25, 0.25, 0.22784810126582278, 0.20253164556962025, 0.20253164556962025, 0.21052631578947367, 0.21052631578947367, 0.19736842105263158, 0.19736842105263158, 0.19736842105263158, 0.19736842105263158, 0.197368

Plot examples which fullfil the criteria

In [18]:
import matplotlib.pyplot as plt


yax=0
indexwithshift=[]
good=[]
for tuplee in pairlist1:
    indexwithshift.append((endswithIndex[PairIndexes[yax]][0],tuplee[0]))
    if dif_bad_normal[yax]>0.65:
        good.append(endswithIndex[PairIndexes[yax]][0])
        yax+=1
        plt.plot((tuplee[0], tuplee[1]), (yax, yax), 'k-')
    else:
        yax+=1
        plt.plot((tuplee[0], tuplee[1]), (yax, yax), 'r-')
print(good)


[38, 22, 54, 70, 30, 23, 62, 20, 51, 53, 76, 61, 49, 46, 4]


Store results in csv files

In [69]:
for ind in indexwithshift:
    dates=[]
    for cycle,unit_nr in zip(dataframes_list[ind[0]]['time_cycles'],dataframes_list[ind[0]]['unit_nr']):
        dates.append( (datetime.datetime(2000, 1, 1) + datetime.timedelta(days=cycle+ind[1])+ datetime.timedelta(seconds=unit_nr)) )
    dataframes_list[ind[0]]['Artficial_timestamp']=dates
    dataframes_list[ind[0]]=dataframes_list[ind[0]].set_index('Artficial_timestamp')
    dataframes_list[ind[0]]=dataframes_list[ind[0]].drop(['unit_nr', 'time_cycles'], axis=1)
    dataframes_list[ind[0]].to_csv('f0001/f01_'+str(ind[0])+'.csv')  
dataframes_list[38].head()

/tmp/ipykernel_5971/929070919.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframes_list[ind[0]]['Artficial_timestamp']=dates


,s_2,s_3,s_4,s_6,s_7,s_8,s_9,s_11,s_12,s_13,s_14,s_15,s_17,s_20,s_21
Artficial_timestamp,,,,,,,,,,,,,,,
2000-01-02 00:00:39,642.72,1592.37,1410.00,21.61,552.82,2388.12,9043.49,47.64,521.78,2388.10,8133.27,8.4451,392,38.90,23.2947
2000-01-03 00:00:39,642.61,1586.25,1407.99,21.61,553.25,2388.10,9049.07,47.60,521.56,2388.10,8126.00,8.4279,395,38.82,23.4604
2000-01-04 00:00:39,642.26,1586.25,1407.48,21.61,552.90,2388.13,9040.05,47.56,522.15,2388.07,8123.32,8.4143,394,38.73,23.2669
2000-01-05 00:00:39,642.46,1587.90,1405.23,21.61,553.55,2388.12,9053.54,47.40,521.83,2388.13,8123.19,8.4599,394,38.82,23.3789
2000-01-06 00:00:39,642.31,1587.08,1408.78,21.61,553.40,2388.10,9044.92,47.50,521.76,2388.07,8125.88,8.4607,394,38.73,23.3338


In [70]:
dataframes_list[39].head()

,s_2,s_3,s_4,s_6,s_7,s_8,s_9,s_11,s_12,s_13,s_14,s_15,s_17,s_20,s_21
Artficial_timestamp,,,,,,,,,,,,,,,
2000-04-12 00:00:40,642.71,1594.14,1412.72,21.61,552.90,2388.08,9051.76,47.59,521.25,2388.07,8134.87,8.4417,394,38.61,23.2017
2000-04-13 00:00:40,642.91,1585.97,1414.86,21.61,553.02,2388.13,9058.39,47.55,521.29,2388.14,8137.99,8.4227,392,38.63,23.3334
2000-04-14 00:00:40,643.19,1593.68,1410.39,21.61,552.70,2388.16,9052.46,47.39,521.49,2388.11,8142.58,8.4526,394,38.81,23.3308
2000-04-15 00:00:40,642.76,1592.30,1397.07,21.61,554.10,2388.13,9053.78,47.65,520.89,2388.11,8136.67,8.4452,393,39.00,23.3082
2000-04-16 00:00:40,642.90,1586.67,1412.14,21.61,553.47,2388.16,9058.23,47.47,520.75,2388.11,8132.89,8.4603,394,38.73,23.3157


In [71]:
indexwithshift=sorted(indexwithshift, key=lambda tup: tup[0])
indexwithshiftSTR=[str(s[0]) for s in indexwithshift]
indexwithshiftSTR.sort()
print(indexwithshiftSTR)

['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '5', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '6', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '7', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '8', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '9', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99']


Plot id of the run to failure examples will bw analyzed.

In [72]:
indexInGrand=[]
for j in good:
    ccc=0
    for i in indexwithshiftSTR:
        if str(j)==i:
            indexInGrand.append(ccc)
        ccc+=1
print(indexInGrand)

[32, 15, 50, 68, 24, 16, 59, 13, 47, 49, 74, 58, 44, 41]
